In [1]:
# основной ROS пакет
import rospy

# пакет для работы с BAG файлами
import rosbag

import os
import cv2

# пакет для преобразования изображений в ROS сообщения и обратно
import cv_bridge

# тип сообщения для изображений
from sensor_msgs.msg import Image

In [2]:
BASE_PATH = '<path to extracted data>'

## Запись изображений в BAG файл

In [3]:
OUTPUT_BAG_PATH = 'output.bag'
INPUT_IMAGE_DIR_PATH = os.path.join(BASE_PATH, '2011_09_26/2011_09_26_drive_0005_sync/image_00/data/')
FREQ = 10
OUTPUT_TOPIC_NAME = '/images'

In [4]:
br = cv_bridge.CvBridge()

In [5]:
with rosbag.Bag(OUTPUT_BAG_PATH, 'w') as bag:
    for i, image_name in enumerate(sorted(os.listdir(INPUT_IMAGE_DIR_PATH))):
        image_path = os.path.join(INPUT_IMAGE_DIR_PATH, image_name)
        image = cv2.imread(image_path)

        timestamp = i / FREQ
        t = rospy.Time.from_sec(timestamp)

        image_msg = br.cv2_to_imgmsg(image)
        image_msg.header.frame_id = 'left_camera'
        image_msg.header.stamp = t

        bag.write(OUTPUT_TOPIC_NAME, image_msg, t)

## Считывание изображений из BAG файла 

In [6]:
INPUT_BAG_PATH = os.path.join(BASE_PATH, 'kitti_2011_09_26_drive_0005_synced.bag')
INPUT_TOPIC_NAME = '/kitti/camera_color_left/image_raw'
OUTPUT_IMAGE_DIR_PATH = 'images'

In [7]:
os.makedirs(OUTPUT_IMAGE_DIR_PATH, exist_ok=True)

In [8]:
br = cv_bridge.CvBridge()

In [9]:
with rosbag.Bag(INPUT_BAG_PATH, 'r') as bag:
    for i, (topic_name, image_msg, t) in enumerate(bag.read_messages(topics=[INPUT_TOPIC_NAME])):
        image = br.imgmsg_to_cv2(image_msg)
        
        image_name = '{:0>8}.png'.format(i)
        image_path = os.path.join(OUTPUT_IMAGE_DIR_PATH, image_name)
        
        cv2.imwrite(image_path, image)